## 1.3 Price Dataset Construction for each Symbol
### Importers

In [ ]:
if "google.colab" in str(get_ipython()):
  print("***********************************")
  print("Google Colab environment deployment")
  print("***********************************")
  !pip install yfinance
  from google.colab import drive
  drive.mount('/content/gdrive')
  print("Google Colab environment deployment")
  root = "gdrive/MyDrive/"
else: # local or Paperspace Gradient
  root = ""

In [ ]:
# General and Particular Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Price Data related
import re
import yfinance as yf
import nasdaqdatalink
nasdaqdatalink.read_key(filename="data/.my_nq_apikey")

from tqdm.auto import tqdm
import time
from datetime import datetime, timedelta
from pandas_datareader import data as pdr
yf.pdr_override()

# Scraping component timeseries (just tested on local envs)
import requests
from bs4 import BeautifulSoup
import json

# Exceptions handling with pkl component_timeseries
import glob
import os

In [ ]:
# request page (does not work on Cloud Platforms)
try: # to be integrated/merged with index_compo_webscraper
    html = requests.get("https://www.blackrock.com/cl/productos/239726/#tabsAll").content
    soup = BeautifulSoup(html)

    # find available dates
    holdings = soup.find("div", {"id": "holdings"})
    dates_div = holdings.find_all("div", "component-date-list")[1]
    dates_div.find_all("option")
    dates = [option.attrs["value"] for option in dates_div.find_all("option")]

    # download constituents for each date
    constituents = pd.Series(dtype=object)
    for date in dates:
        resp = requests.get(
            f"https://www.blackrock.com/cl/productos/239726/239726/ishares-core-sp-500-etf/1506433277024.ajax?tab=all&fileType=json&asOfDate={date}"
        ).content[3:]
        tickers = json.loads(resp)
        tickers = [(arr[0], arr[1]) for arr in tickers['aaData']]
        date = datetime.strptime(date, "%Y%m%d")
        constituents[date] = tickers

    constituents = constituents.iloc[::-1] # reverse into cronlogical order
    constituents.head()
    constituents.to_pickle("data/pkl/sp500_component_timeseries.pkl")
except:    
    list_of_pkl_files = glob.glob('data/pkl/IVV*') # * means all if need specific format then *.csv
    latest_file = max(list_of_pkl_files, key=os.path.getctime)
    constituents = pd.read_pickle(root + latest_file)

In [ ]:
try:
    first_components_date = constituents.index[0]
    # we substract one-year to start construction of RPMs from this point (we assume static composition from then)
    first_date = first_components_date-timedelta(days=365) 
    first_date = first_date.strftime("%Y-%m-%d")
    """
    https://data.nasdaq.com/databases/WIKIP/documentation?anchor=overview
    Note: As of April 11, 2018 this data feed is no longer actively supported by the Quandl 
    community. We will continue to host this data feed on Quandl, 
    but we do not recommend using it for investment or analysis.
    """
    wikip_last_quarter_supported = "2018-03-31"

    nasdaqdatalink.export_table('WIKI/PRICES', 
        qopts = {'columns': ['ticker', 'date', 'adj_close', 'adj_volume']}, 
        date = {'gte': first_date, 'lte': wikip_last_quarter_supported},
        filename = "data/" + first_date.replace("-","") + "_WIKI_PRICES.zip"
    )    
finally:
    lst_of_zip_wikip_files = glob.glob('data/*WIKI_PRICES*zip')
    latest_file = max(lst_of_zip_wikip_files, key=os.path.getctime)
    wiki = pd.read_csv(root + latest_file, parse_dates=True)    
   
wiki = dict(tuple(wiki.groupby('ticker')))
for ticker in wiki:
    wiki[ticker].set_index("date", inplace=True)

In [25]:
pdr.get_data_yahoo?

Signature:
pdr.get_data_yahoo(
    tickers,
    start=None,
    end=None,
    actions=False,
    threads=True,
    group_by='column',
    auto_adjust=False,
    back_adjust=False,
    progress=True,
    period='max',
    show_errors=True,
    interval='1d',
    prepost=False,
    proxy=None,
    rounding=False,
    timeout=None,
    **kwargs,
)
Docstring:
Download yahoo tickers
:Parameters:
    tickers : str, list
        List of tickers to download
    period : str
        Valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
        Either Use period parameter or use start and end
    interval : str
        Valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
        Intraday data cannot extend last 60 days
    start: str
        Download start date string (YYYY-MM-DD) or _datetime.
        Default is 1900-01-01
    end: str
        Download end date string (YYYY-MM-DD) or _datetime.
        Default is now
    group_by : str
        Group by 'ticker' or 'column' (default)
 

In [30]:
import time
import re

yfinance_exceptions = ['GAS', 'HAR', 'CSC', 'EMC', 'GME', 'POM', 'PCL', 'COV', 'ETN', 'COL', 'LLL']
nasdaqdl_exceptions = ['IR']

def yfinance_data(ticker, start, end):    
    ticker = change_ticker(ticker)
    # abnormal quotes from yfinance or bad mixing with nasdaqdl
    if ticker in yfinance_exceptions: 
        return None    
    try:
        df = pdr.get_data_yahoo(ticker, start, end, progress=False, show_errors=False, timeout=0.5) 
    except:
        time.sleep(1)
        try:            
            df = pdr.get_data_yahoo(ticker, start, end, progress=False, show_errors=False, timeout=0.5)
        except:
            return None
    # adjust close using adj close
    adjfactor = df["Close"] / df["Adj Close"]    
    df["Close"] = df["Adj Close"]
    df["Volume"] *= adjfactor
    df = df.drop(["Adj Close", "Open", "High", "Low"], axis=1)
    df = df.rename(str.lower, axis='columns')
    df.index.rename('date', inplace=True)    
    return df

def nasdaqdl_data(ticker, start, end):    
    # abnormal quotes from yfinance or bad mixing with nasdaqdl (because of splittings a posteriori)
    if ticker in nasdaqdl_exceptions: 
        return None    
    if ticker in wiki:
        df = wiki[ticker][start:end]
    else:
        ticker = change_ticker(ticker)
        if ticker in wiki:
            df = wiki[ticker][start:end]
        else:
            return None
    df = df.drop(['ticker'], axis=1)
    df = df.rename(index=str, columns={"adj_close": "close","adj_volume": "volume"})
    return df

def change_ticker(ticker):
    rename_table = {
        "-": "LPRAX", # BlackRock LifePath Dynamic Retirement Fund
        "8686": "AFL", # AFLAC
        "4XS": "ESRX", # Express Scripts Holding Company 
        "AAZ": "APC", # Anadarko Petroleum Corporation
        "AG4": "AGN", # Allergan plc
        "BFB": "BF-B", # Brown-Forman Corporation
        "BF.B": "BF-B", # Brown-Forman Corporation
        "BF/B": "BF-B", # Brown-Forman Corporation
        "BF_B" : "BF-B", # Brown-Forman Corporation
        "BLD WI": "BLD", # TopBuild Corp.
        "BRKB": "BRK-B", # Berkshire Hathaway Inc.
        "BRK_B": "BRK-B", # Berkshire Hathaway Inc.
        "CC WI": "CC", # The Chemours Company
        "DC7": "DFS", # Discover Financial Services
        "DWDP": "DD", # Discover Financial Services
        "FB": "META",   # Facebook
        "GGQ7": "GOOG", # Alphabet Inc. Class C
        "GEC": "GE", # General Electric
        "HNZ": "KHC", # The Kraft Heinz Company
        "INCO": "INTC", # Intel
        "LOM": "LMT", # Lockheed Martin Corp.
        "LTD": "LB", # L Brands Inc.
        "LTR": "L", # Loews Corporation        
        "MPN": "MPC", # Marathon Petroleum Corp.
        "MYL": "VTRS", # Mylan NV (VIATRIS)
        "MWZ": "MET", # Metlife Inc.
        "MX4A": "CME", # CME Group Inc.
        "NCRA": "NWSA", # News Corporation
        "NTH": "NOC", # Northrop Grumman Crop.
        "PA9": "TRV", # The Travelers Companies, Inc.
        "QCI": "QCOM", # Qualcomm Inc.
        "RN7": "RF", # Regions Financial Corp
        "RTN" : "RTX", # Raytheon
        "SLBA": "SLB", # Schlumberger Limited
        "SYF-W": "SYF", # Synchrony Financial
        "SWG": "SCHW", # The Charles Schwab Corporation 
        "UAC/C": "UAA", # Under Armour Inc Class A
        "UBSFT": "UBSFY", # Ubisoft Entertainment
        "USX1": "X", # United States Steel Corporation
        "UUM": "UNM", # Unum Group
        "VISA": "V", # Visa Inc         
        "VIAC": "VIA", # viacom
        "WLTW": "WTW" # Willis Towers Watson                
    }
    if ticker in rename_table:
        fix = rename_table[ticker]
    else:
        fix = re.sub(r'[^A-Z]+', '', ticker)
    return fix

In [ ]:
from tqdm.auto import tqdm
data = {}
skips = set()

#constituents = pd.read_pickle(root + latest_file)
constituents = constituents["2011-06-01":]
"""
Fast Check for Ticker Price Data Availability
"""    
#subsetting = ["LLL", "AIV", "VIA", "IR", "LUMN"]                
subsetting = []                
global_excludes = ["CBE", "TIE", "BMC"]   # bad quotes in both sources (or manually downloaded)
# Y! manually downloaded ["AIV", "EQT"]

query = []
dates = []
for i in tqdm(range(0, len(constituents) - 1)):
    start = str(constituents.index[i].date())
    end = str((constituents.index[i + 1].to_pydatetime() - timedelta(days=1)).date())
    tickers_supported = 0    
    for counter, company in enumerate(constituents[i]):        
        if change_ticker(company[0]) not in subsetting and len(subsetting)!=0:       
            continue        
        if company[0] in global_excludes: # Company with bad behaviour for Quotes in Y! Finance (TBR)
            continue
        if company in skips:
            continue        
        """
        #df = nasdaqdl_data(company[0], start, end)    
        try:
            if df.empty:
            df = yfinance_data(company[0], start, end);                
        except:
            if df is None:
            df = yfinance_data(company[0], start, end);                
        """
        df = yfinance_data(company[0], start, end);                
        if df is None or df.empty:
            df = nasdaqdl_data(company[0], start, end);                
        if df is None or df.empty:
            skips.add(company)
            continue        
        if change_ticker(company[0]) in data:
            data[change_ticker(company[0])]  = pd.concat([data[change_ticker(company[0])],df])
        else:
            data[change_ticker(company[0])] = df        
        tickers_supported+=1
    print(f"{start}, tickers cached {tickers_supported}")    
    query.append(tickers_supported)        
    dates.append(start) 
print(skips)   

  0%|          | 0/129 [00:00<?, ?it/s]

2011-06-30, tickers cached 475
{('WYND', 'TRAVEL LEISURE INC'), ('WFR', 'MEMC ELECTRONIC MATERIALS INC.'), ('RSHCQ', 'RS LEGACY CORP'), ('LIFE', 'LIFE TECHNOLOGIES CORP'), ('NSM.2', 'NATIONAL SEMICONDUCTOR CORP'), ('AABA', 'ALTABA INC'), ('NVLS', 'NOVELLUS SYSTEMS INC.'), ('CPPRQ', 'OLD COPPER INC'), ('HNZ', 'H.J. HEINZ CO.'), ('DEN', 'DENBURY INC'), ('MI', 'MARSHALL & ILSLEY'), ('MMI', 'MOTOROLA MOBILITY HOLDINGS INC.'), ('PGN', 'PROGRESS ENERGY INC.'), ('CVH', 'COVENTRY HEALTH CARE INC'), ('PCS', 'METROPCS COMMUNICATIONS INC.'), ('CEG', 'CONSTELLATION ENERGY GROUP INC.'), ('SLE', 'SARA LEE CORP.'), ('SUN', 'SUNOCO INC.'), ('CEPH', 'CEPHALON INC'), ('DOFSQ', 'DIAMOND OFFSHORE DRILLING INC'), ('NCRA', 'NEWS CORP. CL A'), ('MHS', 'MEDCO HEALTH SOLUTIONS INC'), ('KFT', 'KRAFT FOODS INC A'), ('ANRZQ', 'ALPHA NATURAL RESOURCES INC')}
2011-07-29, tickers cached 477
{('WYND', 'TRAVEL LEISURE INC'), ('WFR', 'MEMC ELECTRONIC MATERIALS INC.'), ('RSHCQ', 'RS LEGACY CORP'), ('LIFE', 'LIFE TECHNOL

In [32]:
pd.DataFrame({"start": dates, "ntickers": query}).to_csv(root + "data/sp500_components_cached.csv", index=False)
skips_df = pd.DataFrame(skips, columns=["ticker", "company"])
skips_df.sort_values(by="ticker").drop_duplicates(subset="ticker").to_csv("data/skips_alltimes.csv", index=False)

In [33]:
for ticker, df in data.items():
    df = df.reset_index().drop_duplicates(subset='date').set_index('date')
    df.to_csv(root + f"data/{change_ticker(ticker)}.csv")
    data[ticker] = df
tickers = [change_ticker(ticker) for ticker in data.keys()]
pd.Series(tickers).to_csv(root + "data/tickers.csv", index=False)

### Other Checks

In [ ]:
baseline_pfolio = [pd.read_csv(f"data/{ticker}.csv", index_col='date', parse_dates=True)['close'].rename(ticker).pct_change() for ticker in tickers]
baseline_pfolio = pd.concat(baseline_pfolio, axis=1, sort=True)

In [ ]:
#lst = []
for i in tqdm(range(0, len(constituents) - 1)):
    for company in constituents[i]:        
        if company[0]=="TER":            
            last =constituents.index[i]
            corp = company
            #lst.append((*corp, last))
            print(corp, last)
            continue

""" rarunos = pd.DataFrame(lst, columns=["ticker", "company_name", "date_in_ivv"])
rarunos.drop_duplicates(subset="date_in_ivv").to_csv("data/rarunos_outlier_max.csv") """

In [ ]:
pdr.get_data_yahoo("LUMN", "2011-06-01", "2022-07-29")["Adj Close"].plot(figsize=(16,9));

In [ ]:
sim_rsp = (
    (pd.concat(
        [pd.read_csv(f"data/{ticker}.csv", index_col='date', parse_dates=True)[
            'close'
        ].pct_change()
        for ticker in tickers],
        axis=1,
        sort=True,
    ).mean(axis=1, skipna=True) + 1)
    .cumprod()
    .rename("SIM")
)

rsp = (
    (pdr.get_data_yahoo("RSP", sim_rsp.index[0], sim_rsp.index[-1])[
        "Adj Close"
    ].pct_change() + 0.002 / 252 + 1)  # 0.20% annual ER
    .cumprod()
    .rename("RSP")
)

sim_rsp.plot(legend=True, title="RSP vs. Un-Survivorship-Biased Strategy", figsize=(12, 9))
rsp.plot(legend=True)
plt.grid();

In [ ]:
pictionary = {ticker: pd.read_csv(root + f"data/{ticker}.csv", index_col='date', parse_dates=True)['close'].pct_change() for ticker in tickers}
df = pd.DataFrame(pictionary)

In [ ]:
df.head()

In [ ]:
(1+df.mean(axis=1, skipna=True)).cumprod().rename("baseline replicating pfolio").plot(legend=True, title="baseline versus RSP ETF", figsize = (16,9))
#rsp[:"2018-04-01"].plot(legend=True);